In [1]:
!echo $CDSW_APP_PORT

8090


In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime

%matplotlib inline

In [3]:
merged_all = pd.read_csv('merged_all.csv')

In [4]:
X = merged_all.drop('Weekly_Sales', axis=1)
y = merged_all['Weekly_Sales']

In [5]:
from sklearn.model_selection import train_test_split
train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)>

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [7]:
import h2o

In [8]:
from h2o.automl import H2OAutoML

In [9]:
h2o.init(max_mem_size='12G', nthreads=4, port=8090)

/home/cdsw/.local/lib/python3.6/site-packages/h2o/backend/connection.py:369: UserWarning: Proxy is defined in the environment: HTTP_PROXY. This may interfere with your H2O Connection.
  "This may interfere with your H2O Connection." % name)
/home/cdsw/.local/lib/python3.6/site-packages/h2o/backend/connection.py:369: UserWarning: Proxy is defined in the environment: http_proxy. This may interfere with your H2O Connection.
  "This may interfere with your H2O Connection." % name)


Checking whether there is an H2O instance running at http://localhost:8090 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-0ubuntu1~16.04.1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /home/cdsw/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplo69pj3j
  JVM stdout: /tmp/tmplo69pj3j/h2o_cdsw_started_from_python.out
  JVM stderr: /tmp/tmplo69pj3j/h2o_cdsw_started_from_python.err
  Server is running at http://127.0.0.1:8094
Connecting to H2O server at http://127.0.0.1:8094 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_cdsw_r1rgp1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10.66 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [10]:
merged_all_train = X_train.join(y_train)

In [11]:
merged_all.head()

,Unnamed: 0,Store,Dept,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size,Day,Month,Year,Holiday
0,0,1,1,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,0.0,211.096358,8.106,False,A,151315,Friday,February,2010,None
1,1,1,1,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,0.0,211.242170,8.106,True,A,151315,Friday,February,2010,Super Bowl
2,2,1,1,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,0.0,211.289143,8.106,False,A,151315,Friday,February,2010,None
3,3,1,1,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,0.0,211.319643,8.106,False,A,151315,Friday,February,2010,None
4,4,1,1,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,0.0,211.350143,8.106,False,A,151315,Friday,March,2010,None


In [12]:
merged_all.columns

Index(['Unnamed: 0', 'Store', 'Dept', 'Weekly_Sales', 'IsHoliday_x',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'Type',
       'Size', 'Day', 'Month', 'Year', 'Holiday'],
      dtype='object')

In [13]:
Xh_train = (merged_all.columns).tolist()
training_frame = h2o.H2OFrame(merged_all)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
auto_ml = H2OAutoML(max_models=10)
auto_ml.train(y='Weekly_Sales', x=merged_all.columns.tolist(), training_frame=training_frame)

AutoML progress: |██

In [ ]:
auto_ml.leaderboard

In [ ]:
bar = sns.barplot(data=auto_ml.leaderboard.as_data_frame(), y='mae', x='model_id')
labels=bar.get_xticklabels()
bar.set_xticklabels(rotation=90, labels=labels)


In [ ]:
h2o.save_model(model=auto_ml.leader, path="/tmp/mymodel", force=True) 